In [3]:
import numpy
import scipy.optimize
import csv

from math import exp
from math import log
from collections import defaultdict
from sklearn import svm

def parseDataFromFile(fname):
    cursor = open(fname,'r')
    csvreader = csv.reader(cursor,delimiter=';')
    return csvreader
      
print "Reading data..."
data = parseDataFromFile("Assignment2/database.csv")
data = list(data)
print "done"


### Preprocessing data ###
crime_data = [l[0].split(",") for l in data[1:]]
race_type = defaultdict(int)

Reading data...


done


In [5]:
########################
#    Define Function   #
########################

# Logistic regression by gradient ascent
# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print "ll =", loglikelihood
  return -loglikelihood


# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

# Inner Product
def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

# Sigmoid function
def sigmoid(x):
  return 1.0 / (1 + exp(-x))

In [7]:
##### Predicting Killer Sex #####

### data initialization ###
month = [0]*12
race = [0]*5
sex = [0]*3
relationship = [0]*27
weapon = [0]*16
crime_type = [0]*2
killer_ethnicity = [0]*3
victim_ethnicity = [0]*3

train_features = []
train_sex = []
train_sex_svm = []
cnt_male = 0
cnt_female = 0


### Data Training ###
for l in crime_data[:20000]:
    if "Yes" in l[10] and ("Male" or "Female" in l[15]):
        weapon = ["Handgun" in l[20], "Knife" in l[20], "Blunt Object" in l[20], "Firearm" in l[20],
                  "Unknown" in l[20], "Shotgun" in l[20], "Rifle" in l[20], "Strangulation" in l[20],
                  "Fire" in l[20], "Suffocation" in l[20], "Gun" in l[20], "Drugs" in l[20],
                  "Drowning" in l[20], "Explosives" in l[20], "Poison" in l[20], "Fall" in l[20]]
        relationship = ["Stepmother" in l[19], "Employee" in l[19], "Employer" in l[19],
                        "Ex-Husband" in l[19], "Stepdaughter" in l[19], "Stepson" in l[19],
                        "Sister" in l[19], "Stepfather" in l[19], "friend" in l[19],
                        "Common-Law Husband" in l[19], "Ex-Wife" in l[19], "Common-Law Wife" in l[19],
                        "In-Law" in l[19], "Mother" in l[19], "Father" in l[19],
                        "Brother" in l[19], "Neighbor" in l[19],
                        "Daughter" in l[19], "Husband" in l[19], "Family" in l[19],
                        "Son" in l[19], "Friend" in l[19], "Wife" in l[19],
                        "Stranger" in l[19], "Acquaintance" in l[19]]
        killer_ethnicity = ["Unknown" in l[18], l[18] == "Not Hispanic", l[18] == "Hispanic"]
        victim_ethnicity = ["Unknown" in l[14], l[14] == "Not Hispanic", l[14] == "Hispanic"]
        victim_race = ["White" in l[13], "Black" in l[13], "Native American" in l[13], 
                       "Asian" in l[13], "Unknown" in l[13]]
        victim_sex = ["Male" in l[11], "Female" in l[11]]
        victim_age = [int(l[12])]
        killer_age = [int(l[16])]
        
        # Know nothing about the killer
        feature_arr = victim_ethnicity + victim_sex + victim_age + victim_race + [1]
        
        # Train whether the killer is a male or a female
        train_sex.append("Male" in l[15])
        train_features.append(feature_arr)
        
        # Using SVM to train
        if "Male" in l[15]:
            train_sex_svm.append(1)
        elif "Female" in l[15]:
            train_sex_svm.append(-1)

In [9]:
# weight training
clf_w = svm.SVC(class_weight = {1: 1, -1: 5})#[weight_male, weight_female]"")
clf_w.fit(train_features, train_sex_svm)
train_sex_predict_svm = clf_w.predict(train_features)
TP = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)])
TN = sum([s[0] == s[1] and s[1] == -1 for s in zip(train_sex_predict_svm, train_sex_svm)])
FP = sum([s[0] != s[1] and s[1] == -1 for s in zip(train_sex_predict_svm, train_sex_svm)])
FN = sum([s[0] != s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)])
TPR = TP * 1.0 / (TP + FN) 
TNR = TN * 1.0 / (TN + FP)
precision = sum([s[0] == s[1] for s in zip(train_sex_predict_svm, train_sex_svm)]) * 1.0 \
                / len(train_sex_predict_svm)
recall = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)]) * 1.0 \
                / sum([s == 1 for s in train_sex_svm])
#F1 = 2 * (precision + recall) * 1.0 / (precision + recall)
print "TPR", TPR
print "TNR", TNR
print "BER", 1 - (TPR + TNR) * 1.0 / 2
#print "F1", F1

TPR 0.718332658843
TNR 0.584867075665
BER 0.348400132746


In [10]:
precision

0.70009083921459014

In [12]:
# oversampling training
from imblearn.over_sampling import SMOTE
X = train_features
y = train_sex_svm
sm = SMOTE(kind='svm')
X_resampled, y_resampled = sm.fit_sample(X, y)
clf_over = svm.SVC(C=0.1)
train_features = X_resampled
train_sex_svm = y_resampled
clf_over.fit(train_features, train_sex_svm)
train_sex_predict_svm = clf_over.predict(train_features)
TP = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)])
TN = sum([s[0] == s[1] and s[1] == -1 for s in zip(train_sex_predict_svm, train_sex_svm)])
FP = sum([s[0] != s[1] and s[1] == -1 for s in zip(train_sex_predict_svm, train_sex_svm)])
FN = sum([s[0] != s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)])
TPR = TP * 1.0 / (TP + FN) 
TNR = TN * 1.0 / (TN + FP)
precision = sum([s[0] == s[1] for s in zip(train_sex_predict_svm, train_sex_svm)]) * 1.0 \
                / len(train_sex_predict_svm)
recall = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict_svm, train_sex_svm)]) * 1.0 \
                / sum([s == 1 for s in train_sex_svm])
#F1 = 2 * (precision + recall) * 1.0 / (precision + recall)
print "TPR", TPR
print "TNR", TNR
print "BER", 1 - (TPR + TNR) * 1.0 / 2
#print "F1", F1

TPR 0.729340348037
TNR 0.712343180898
BER 0.279158235532


In [13]:
precision

0.72084176446782677

In [28]:
### Data Validating ###
valid_features = []
valid_sex = []
valid_sex_svm = []

for l in crime_data[200000:500000]:
    if "Yes" in l[10] and ("Male" or "Female" in l[15]):
        weapon = ["Handgun" in l[20], "Knife" in l[20], "Blunt Object" in l[20], "Firearm" in l[20],
                  "Unknown" in l[20], "Shotgun" in l[20], "Rifle" in l[20], "Strangulation" in l[20],
                  "Fire" in l[20], "Suffocation" in l[20], "Gun" in l[20], "Drugs" in l[20],
                  "Drowning" in l[20], "Explosives" in l[20], "Poison" in l[20], "Fall" in l[20]]
        relationship = ["Stepmother" in l[19], "Employee" in l[19], "Employer" in l[19],
                        "Ex-Husband" in l[19], "Stepdaughter" in l[19], "Stepson" in l[19],
                        "Sister" in l[19], "Stepfather" in l[19], "friend" in l[19],
                        "Common-Law Husband" in l[19], "Ex-Wife" in l[19], "Common-Law Wife" in l[19],
                        "In-Law" in l[19], "Mother" in l[19], "Father" in l[19],
                        "Brother" in l[19], "Neighbor" in l[19],
                        "Daughter" in l[19], "Husband" in l[19], "Family" in l[19],
                        "Son" in l[19], "Friend" in l[19], "Wife" in l[19],
                        "Stranger" in l[19], "Acquaintance" in l[19]]
        killer_ethnicity = ["Unknown" in l[18], l[18] == "Not Hispanic", l[18] == "Hispanic"]
        victim_ethnicity = ["Unknown" in l[14], l[14] == "Not Hispanic", l[14] == "Hispanic"]
        victim_race = ["White" in l[13], "Black" in l[13], "Native American" in l[13], 
                       "Asian" in l[13], "Unknown" in l[13]]
        victim_sex = ["Male" in l[11], "Female" in l[11]]
        victim_age = [int(l[12])]
        killer_age = [int(l[16])]
        
        # Know nothing about the killer
        feature_arr_valid = victim_ethnicity + victim_sex + victim_age  + victim_race + [1]
        # Train whether the killer is a male or a female
        valid_sex.append("Male" in l[15])
        valid_features.append(feature_arr_valid)
        
        # Using SVM to train
        if "Male" in l[15]:
            valid_sex_svm.append(1)
        elif "Female" in l[15]:
            valid_sex_svm.append(-1)

In [30]:
# weight valid
valid_sex_predict_svm = clf_w.predict(valid_features)
TP = sum([s[0] == s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
TN = sum([s[0] == s[1] and s[1] == -1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
FP = sum([s[0] != s[1] and s[1] == -1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
FN = sum([s[0] != s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
TPR = TP * 1.0 / (TP + FN) 
TNR = TN * 1.0 / (TN + FP)
precision = sum([s[0] == s[1] for s in zip(valid_sex_predict_svm, valid_sex_svm)]) * 1.0 \
                / len(valid_sex_predict_svm)
recall = sum([s[0] == s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)]) * 1.0 \
                / sum([s == 1 for s in valid_sex_svm])
F1 = 2 * (precision + recall) * 1.0 / (precision + recall)
print "Weighting method"
print "TPR", TPR
print "TNR", TNR
print "BER", 1 - (TPR + TNR) * 1.0 / 2
#print "F1", F1

Weighting method
TPR 0.716152569409
TNR 0.475774857426
BER 0.404036286583


In [12]:
# oversampling valid
valid_sex_predict_svm = clf_over.predict(valid_features)
TP = sum([s[0] == s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
TN = sum([s[0] == s[1] and s[1] == -1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
FP = sum([s[0] != s[1] and s[1] == -1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
FN = sum([s[0] != s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)])
TPR = TP * 1.0 / (TP + FN) 
TNR = TN * 1.0 / (TN + FP)
precision = sum([s[0] == s[1] for s in zip(valid_sex_predict_svm, valid_sex_svm)]) * 1.0 \
                / len(valid_sex_predict_svm)
recall = sum([s[0] == s[1] and s[1] == 1 for s in zip(valid_sex_predict_svm, valid_sex_svm)]) * 1.0 \
                / sum([s == 1 for s in valid_sex_svm])
F1 = 2 * (precision + recall) * 1.0 / (precision + recall)
print "Oversampling method"
print "TPR", TPR
print "TNR", TNR
print "BER", 1 - (TPR + TNR) * 1.0 / 2
#print "F1", F1

Oversampling method
TPR 0.902159855147
TNR 0.229456979916
BER 0.434191582468


In [12]:
from imblearn.over_sampling import SMOTE
X = train_features
y = train_sex
sm = SMOTE(kind='svm')
X_resampled, y_resampled = sm.fit_sample(X, y)

# Logestic Regression
lam = 0.1
theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(feature_arr), fprime, pgtol=10, 
                                         args=(X_resampled, y_resampled, lam))

In [22]:
train_sex_predict = [sigmoid(inner(theta, x)) > 0.5 for x in train_features]
train_correct = [(a == b) for (a,b) in zip(train_sex_predict,train_sex)]
train_accuracy = sum(train_correct) * 1.0 / len(train_correct)
print "Training: Accuracy of Killer sex given only victim info = ", train_accuracy


TP = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict, train_sex)])
TN = sum([s[0] == s[1] and s[1] == 0 for s in zip(train_sex_predict, train_sex)])
FP = sum([s[0] != s[1] and s[1] == 0 for s in zip(train_sex_predict, train_sex)])
FN = sum([s[0] != s[1] and s[1] == 1 for s in zip(train_sex_predict, train_sex)])
TPR = TP * 1.0 / (TP + FN) 
TNR = TN * 1.0 / (TN + FP)
precision = sum([s[0] == s[1] for s in zip(train_sex_predict, train_sex)]) * 1.0 \
                / len(train_sex_predict)
recall = sum([s[0] == s[1] and s[1] == 1 for s in zip(train_sex_predict, train_sex)]) * 1.0 \
                / sum([s == 1 for s in train_sex])
#F1 = 2 * (precision + recall) * 1.0 / (precision + recall)
print "TPR", TPR
print "TNR", TNR
print "BER", 1 - (TPR + TNR) * 1.0 / 2

Training: Accuracy of Killer sex given only victim info =  0.636223883726
TPR 0.681019830028
TNR 0.35327198364
BER 0.482854093166


In [23]:
precision

0.6362238837258053